In [1]:
!pip install -q ultralytics

In [33]:
import os
import json
import math
import torch
import numpy as np
import matplotlib.pyplot as plt

import cv2
from PIL import Image

from ultralytics import YOLO

In [32]:
def get_central_band(image_path, band_width):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculer la position centrale de la bande
    start_x = (width - band_width) // 2
    end_x = start_x + band_width

    # Extraire la bande centrale de l'image
    central_band = image[:, start_x:end_x]

    return central_band

In [34]:
def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)

    return image_src

In [35]:
def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

In [36]:
def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'

    if with_plot:
        fig = plt.figure(figsize=(10, 20))

        ax1 = fig.add_subplot(2, 2, 1)
        ax1.axis("off")
        ax1.title.set_text('Original')
        ax2 = fig.add_subplot(2, 2, 2)
        ax2.axis("off")
        ax2.title.set_text("Equalized")

        ax1.imshow(image_src, cmap=cmap_val)
        ax2.imshow(image_eq, cmap=cmap_val)
        return True, image_eq
    return image_eq

In [37]:
def adjust_brightness_contrast(image, alpha, beta):
    return cv2.addWeighted(image, alpha, image, 0, beta)

In [44]:
def blur_img(img_path):
    maxblur = 9
    ramppow = 1

    # read the input
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    center = [height//2, width//2]

    # blur the image to the maximum desired
    blur = cv2.GaussianBlur(img, (maxblur,maxblur), sigmaX=4000, sigmaY=4000)

    # Note [:, None] and [None, :] specify which dimension that array will specify
    # Linear ramps are between 0 and 1 and have number of increments of width or height
    x = np.linspace(0, 1, width)[:, None]
    y = np.linspace(0, 1, height)[None, :]

    # normalize the center coordinates between 0 and 1
    cx = center[1]/width
    cy = center[0]/height

    # Set graylevel of gradient proportional to distance from center
    # Note multiply by sqrt(2) so 0 to 255 goes diagonally from center
    # Include non-linear ramp power function to blur faster as get farther from the center
    # Linear ramp would be ramppow=1
    ramppow = 1/ramppow
    gradient = np.sqrt((x-cx)**2 + (y-cy)**2)
    gradient = cv2.pow(gradient,ramppow)

    gradient = (255*math.sqrt(2)*gradient).clip(0,255).astype(np.uint8)
    gradient = np.transpose(gradient)
    gradient = cv2.merge([gradient,gradient,gradient])

    # Invert the gradient
    # gradient = 1 - gradient

    # merge the input and blurred image using the gradient
    result = ((gradient*blur.astype(np.float64) + (1-gradient)*img.astype(np.float64))/255).clip(0,255).astype(np.uint8)
    
    return result

In [41]:
def mask_blur(results, bright_path, blur_path):
    
    mask_wm = results[0].masks.data[0]

    image = cv2.imread(bright_path)
    image_blur = cv2.imread(blur_path)

    # Redimensionner le tensor du masque pour qu'il corresponde aux dimensions de l'image
    image_shape = (np.array(image).shape[0], np.array(image).shape[1])
    
    mask_tensor = torch.nn.functional.interpolate(mask_wm.unsqueeze(0).unsqueeze(0), size=image_shape, mode='nearest')
    mask_tensor = mask_tensor.squeeze()

    # Convertir le tensor en un masque binaire numpy
    mask_binary_np = (mask_tensor.cpu().detach().numpy() > 0.5).astype(np.uint8)

    # Convertir les images en tableaux NumPy
    image_np = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_blur_np = cv2.cvtColor(image_blur, cv2.COLOR_BGR2RGB)

    # Appliquer le masque binaire sur l'image floue
    masked_image = np.copy(image_blur_np)
    masked_image[mask_binary_np == 1] = image_np[mask_binary_np == 1]
    
    return masked_image

In [46]:
folder_images = './clean-mannequin'
output_folder = './'

# Load a model
model = YOLO('yolov8x-seg.pt')

band_width = 3000  # Largeur de la bande centrale à extraire

for file in sorted(os.listdir(folder_images)):
    filename = os.path.join(folder_images, file)
    print(filename)
    
    central_band = get_central_band(filename, band_width)
    central_band = cv2.cvtColor(central_band, cv2.COLOR_BGR2RGB)
    
    box_path = os.path.join(output_folder, file).replace(".png", "") + "_box.png"
    cv2.imwrite(box_path, central_band)
    print(box_path)
    
    image_contrast = equalize_this(image_file=box_path, with_plot=False)
    
    contrast_path = box_path.replace(".png", "") + "_contrasted.png"
    cv2.imwrite(contrast_path, image_contrast)
    print(contrast_path)
    
    # Adjust brightness and contrast
    bright_image = adjust_brightness_contrast(image_contrast, 0.9, -20)
    
    bright_path = contrast_path.replace(".png", "") + "_bright.png"
    cv2.imwrite(bright_path, bright_image)
    print(bright_path)
    
    blur_image = blur_img(bright_path)
    
    blur_path = bright_path.replace(".png", "") + "_blur.png"
    cv2.imwrite(blur_path, blur_image)
    print(blur_path)
    
    # Predict with the model
    results = model(bright_path)
    
    final_image = mask_blur(results, bright_path, blur_path)
    
    final_path = blur_path.replace(".png", "") + "_final.png"
    cv2.imwrite(final_path, final_image)
    print(final_path)

/kaggle/input/clean-mannequin/1_out.png
/kaggle/working/1_out_box.png
/kaggle/working/1_out_box_contrasted.png
/kaggle/working/1_out_box_contrasted_bright.png
/kaggle/working/1_out_box_contrasted_bright_blur.png

image 1/1 /kaggle/working/1_out_box_contrasted_bright.png: 640x448 3 persons, 1 handbag, 1994.8ms
Speed: 3.4ms preprocess, 1994.8ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 448)
/kaggle/working/1_out_box_contrasted_bright_blur_final.png
/kaggle/input/clean-mannequin/2_out.png
/kaggle/working/2_out_box.png
/kaggle/working/2_out_box_contrasted.png
/kaggle/working/2_out_box_contrasted_bright.png
/kaggle/working/2_out_box_contrasted_bright_blur.png

image 1/1 /kaggle/working/2_out_box_contrasted_bright.png: 640x448 3 persons, 1944.4ms
Speed: 3.9ms preprocess, 1944.4ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 448)
/kaggle/working/2_out_box_contrasted_bright_blur_final.png
/kaggle/input/clean-mannequin/3_out.png
/kaggle/working/3_out_box.png
/ka